<a href="https://colab.research.google.com/github/mcucii/TextToImageGeneration_StackGAN/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/RIProject')

In [3]:
import config as cfg
from dataset import TextImageDataset
from stage1 import GANTrainer_stage1
from stage2 import GANTrainer_stage2

In [4]:
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch

In [5]:
class Args:
    train = 'y'
    stage = 1

if Args.stage == 1:
    cfg.IMG_SIZE = 64
    cfg.NET_G = ''
    cfg.NET_D = ''
else:
    cfg.IMG_SIZE = 256
    cfg.STAGE1_G = "data_reduced/birds/Model_stage1/netG_epoch_100.pth"
    cfg.STAGE1_D = "data_reduced/birds/Model_stage1/netD_epoch_100.pth"

cfg.STAGE = Args.stage

if Args.train == "y":
    cfg.TRAIN = True
else:
    cfg.NET_G = "../data_reduced/birds/Model_stage2/netG_epoch_100.pth"
    cfg.NET_D = "../data_reduced/birds/Model_stage2/netD_epoch_100.pth"
    cfg.TRAIN = False

if torch.cuda.is_available():
  cfg.DEVICE = 'cuda'
else:
  cfg.DEVICE = 'cpu'

output_dir = '../output/birds'

In [18]:

def train():
  image_transform = transforms.Compose([
      transforms.RandomResizedCrop(cfg.IMG_SIZE),  # Randomly crop and resize the image
      transforms.RandomHorizontalFlip(),         # Randomly flip the image horizontally
      transforms.ColorJitter(),                  # Randomly adjust brightness, contrast, saturation, and hue
      transforms.ToTensor(),                     # Convert the image to a PyTorch tensor
      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize the image
  ])

  image_transform2 = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.1),  # Minimalna šansa za horizontalno ogledanje slike
    transforms.ColorJitter(brightness=0.05, contrast=0.05, saturation=0.05, hue=0.02),  # Vrlo mala promena osvetljenosti, kontrasta, saturacije i nijanse
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),  # Normalizacija slike
  ])

  image_transform3 = transforms.Compose([
            transforms.RandomCrop(cfg.IMG_SIZE),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

  dataset = TextImageDataset(cfg.DATA_DIR, 'train', input_transform=image_transform3)
  dataset_size = len(dataset)
  #print(f"Veličina skupa podataka: {dataset_size}")

  dataloader = DataLoader(dataset, batch_size=cfg.TRAIN_BATCH_SIZE, drop_last=True, shuffle=False, num_workers=4)
  batch_size = dataloader.batch_size
  #print(f"Veličina batch-a: {batch_size}")

  num_batches_per_epoch = len(dataloader)
  #print(f"Broj batch-eva po epohi: {num_batches_per_epoch}")

  if Args.stage == 1:
    trainer = GANTrainer_stage1(output_dir)
  else:
    trainer = GANTrainer_stage2(output_dir)

  trainer.train(dataloader)

In [19]:
def test():
  # TODO
  pass

In [20]:
def main():

  if Args.train == 0:
    cfg.TRAIN = False  # True by default (train)
    # todo: test
    #cfg.NET_G = "?"  # NET_G -> pre-trained model:
  else:
    cfg.TRAIN = True

  if Args.stage == 1:
    cfg.IMG_SIZE = 64
    cfg.STAGE = 1
  else :
    cfg.IMG_SIZE = 256
    cfg.STAGE = 2

  output_dir = '../output/birds'


  if cfg.TRAIN:
    train()
  # else:
  #   test()


In [21]:
main()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch [0/200], Step [0/8], Generator Loss: 4.367394924163818, Discriminator Loss: 1.9060834646224976
Epoch [0/200], Step [1/8], Generator Loss: 1.453080177307129, Discriminator Loss: 2.4879467487335205
Epoch [0/200], Step [2/8], Generator Loss: 1.446690320968628, Discriminator Loss: 1.4992995262145996
Epoch [0/200], Step [3/8], Generator Loss: 2.7509360313415527, Discriminator Loss: 1.59871244430542
Epoch [0/200], Step [4/8], Generator Loss: 1.9393819570541382, Discriminator Loss: 1.9710650444030762
Epoch [0/200], Step [5/8], Generator Loss: 2.4629108905792236, Discriminator Loss: 1.7713828086853027
Epoch [0/200], Step [6/8], Generator Loss: 2.302366018295288, Discriminator Loss: 1.5027687549591064
Epoch [0/200], Step [7/8], Generator Loss: 2.76003098487854, Discriminator Loss: 1.583848476409912
Save G/D models
Epoch [1/200], Step [0/8], Generator Loss: 2.127232551574707, Discriminator Loss: 1.6434741020202637
Epoch [1/200], Step [1/8], Generator Loss: 3.0548651218414307, Discriminator